<a href="https://colab.research.google.com/github/leshaz44/Aliaksei/blob/main/Task_3/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import sqlite3

# Загрузка файла CSV
df = pd.read_csv('data_Salary2.csv')

# Просмотр первых пяти строк загруженного DataFrame
#print(df.head())

try:
    df = pd.read_csv('data_Salary2.csv')  # Читаем CSV-файл

    if len(df) > 0:               # Проверяем наличие записей в таблице
        print("Файл CSV успешно загружен.")
    else:
        print("Файл пустой!")
except FileNotFoundError:
    print("Ошибка: Файл data_Salary2.csv не найден.")
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")

# Подключение к базе данных SQLite
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Создание таблицы
cursor.execute('''CREATE TABLE IF NOT EXISTS table_name (
  id INTEGER PRIMARY KEY,
  Age INTEGER TEXT,
  Gender TEXT,
  Education TEXT,
  Job TEXT,
  Experience FLOAT,
  Salary FLOAT)''')

# Импортируем данные из DataFrame в созданную таблицу
df.to_sql('table_name', con=conn, index=False, if_exists='append')

# Запрос данных
cursor.execute("SELECT * FROM table_name")
print(cursor.fetchall())

# Закрытие соединения
conn.commit()
conn.close()

Файл CSV успешно загружен.
[(1, 32, 'Male', "Bachelor's", 'Software Engineer', 5.0, 90000.0), (2, 28, 'Female', "Master's", 'Data Analyst', 3.0, 65000.0), (3, 45, 'Male', 'PhD', 'Senior Manager', 15.0, 150000.0), (4, 36, 'Female', "Bachelor's", 'Sales Associate', 7.0, 60000.0), (5, 52, 'Male', "Master's", 'Director', 20.0, 200000.0), (6, 29, 'Male', "Bachelor's", 'Marketing Analyst', 2.0, 55000.0), (7, 42, 'Female', "Master's", 'Product Manager', 12.0, 120000.0), (8, 31, 'Male', "Bachelor's", 'Sales Manager', 4.0, 80000.0), (9, 26, 'Female', "Bachelor's", 'Marketing Coordinator', 1.0, 45000.0), (10, 38, 'Male', 'PhD', 'Senior Scientist', 10.0, 110000.0), (11, 29, 'Male', "Master's", 'Software Developer', 3.0, 75000.0), (12, 48, 'Female', "Bachelor's", 'HR Manager', 18.0, 140000.0), (13, 35, 'Male', "Bachelor's", 'Financial Analyst', 6.0, 65000.0), (14, 40, 'Female', "Master's", 'Project Manager', 14.0, 130000.0), (15, 27, 'Male', "Bachelor's", 'Customer Service Rep', 2.0, 40000.0), (16